In [ ]:
!conda install -c conda-forge onnx

In [9]:
import torch
import onnx

model = onnx.load('novianty2018.onnx')
model

TypeError: __init__() got an unexpected keyword argument 'serialized_options'

In [ ]:

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
onnx.helper.printable_graph(model.graph)